# 🧭 Ajout des métadonnées pour assurer:

    🔍 Filtrage intelligent : pour restreindre les résultats à un thème, un niveau de lecture, ou une source spécifique.
    🧠 Contexte enrichi : Le LLM peut utiliser les métadonnées pour formuler des réponses plus nuancées.
    📚 Traçabilité : Permet de retrouver facilement l’origine d’un chunk (livre, page, chapitre…).
    🧪 Benchmarking : pour analyser la performance du RAG par source ou par type de contenu.

🔹 Métadonnées automatiques (déjà disponibles ou déductibles)

In [3]:
import pandas as pd
import re

# Charger les chunks
df = pd.read_csv("../data/ready_for_embedding/chunks.csv")

# Ajouter des métadonnées automatiques
df["length"] = df["text"].apply(lambda x: len(x.split()))
df["theme"] = "à annoter"
df["niveau"] = "à annoter"
df["type"] = "à annoter"

# Sauvegarder pour annotation manuelle ou collaborative
df.to_csv("../data/ready_for_embedding/chunks_with_metadata.csv", index=False)
# Aperçu
df.head()

,source,page,chunk_id,text,length,theme,niveau,type
0,100 idees pour enseigner les ha - Mehdi Liratni,2,100 idees pour enseigner les ha - Mehdi Liratn...,Dr Mehdi Liratni Préface du Pr René Pry 100 ID...,18,à annoter,à annoter,à annoter
1,100 idees pour enseigner les ha - Mehdi Liratni,3,100 idees pour enseigner les ha - Mehdi Liratn...,La version numérique de cet ouvrage a été réal...,30,à annoter,à annoter,à annoter
2,100 idees pour enseigner les ha - Mehdi Liratni,4,100 idees pour enseigner les ha - Mehdi Liratn...,SOMMAIRE Préface Remerciements INTRODUCTION HA...,171,à annoter,à annoter,à annoter
3,100 idees pour enseigner les ha - Mehdi Liratni,5,100 idees pour enseigner les ha - Mehdi Liratn...,CHAPITRE II COMPÉTENCES SOCLES DU COMPORTEMENT...,212,à annoter,à annoter,à annoter
4,100 idees pour enseigner les ha - Mehdi Liratni,6,100 idees pour enseigner les ha - Mehdi Liratn...,Idée 26 Les 4 règles et objectifs importants d...,236,à annoter,à annoter,à annoter


In [4]:
df.tail()

,source,page,chunk_id,text,length,theme,niveau,type
4983,Situations sociales pour les adolescents ebook,131,Situations sociales pour les adolescents ebook...,Étapes de l'analyse de situations sociales à t...,21,à annoter,à annoter,à annoter
4984,Situations sociales pour les adolescents ebook,132,Situations sociales pour les adolescents ebook..., ____________________________________________...,200,à annoter,à annoter,à annoter
4985,Situations sociales pour les adolescents ebook,132,Situations sociales pour les adolescents ebook...,1. Variété de boissons. 2. Les invités n'aimen...,174,à annoter,à annoter,à annoter
4986,Situations sociales pour les adolescents ebook,133,Situations sociales pour les adolescents ebook...,______________________________________________...,143,à annoter,à annoter,à annoter
4987,Situations sociales pour les adolescents ebook,134,Situations sociales pour les adolescents ebook...,"Le livre intitulé ""Situations sociales pour le...",110,à annoter,à annoter,à annoter


🔸 Métadonnées semi-automatiques (à enrichir)

1. 🔍 Déduction par règles simples (automatisable): création des fonctions qui détectent des mots-clés ou des structures typiques dans le texte.

In [5]:
# Créer la fonction à appliquer aux chunks de texte dans la colonne "theme"

def detect_theme(text):
    themes = {
        "émotions": ["joie", "bonheur", "tristesse", "colère", "peur", "émotion", "ressenti", "surprise", "dégout", "anxiété", "stress", "soulagement", "calme", "frustration", "excitation", "déception", "sérénité", "cynisme", "jalousie", "envie", "fierté", "honte", "culpabilité"],
        "relations sociales": ["ami", "groupe", "saluer", "conversation", "interaction", "relation", "social", "copain", "famille", "collègue", "copine", "voisin", "équipe", "communauté", "réseau", "partenaire", "conflit", "coopération", "empathie", "écoute"],
        "autisme": ["asperger", "autiste", "neurodiversité", "diagnostic", "comportement", "spectre", "sensibilité", "stéréotypie", "inclusion", "adaptation", "thérapie", "intervention", "soutien", "accompagnement", "différence", "perception", "communication", "interaction sociale"],
        "vie quotidienne": ["école", "lycée", "collège", "travail", "repas", "transport", "routine", "habitude", "tâche", "activité", "loisir", "temps libre", "organisation", "planification", "gestion du temps", "procrastination", "motivation", "énergie", "sommeil", "hygiène"],
        "développement personnel": ["confiance en soi", "estime de soi", "assertivité", "communication", "gestion du stress", "pleine conscience", "méditation", "objectifs", "motivation", "résilience", "adaptabilité", "créativité", "curiosité", "apprentissage", "croissance", "épanouissement", "bien-être", "équilibre", "satisfaction", "réalisation"],
        "santé mentale": ["anxiété", "dépression", "stress", "burnout", "trouble", "émotion", "pensée", "comportement", "thérapie", "psychiatre", "psychologue", "médication", "soutien", "résilience", "coping", "bien-être", "auto-soin", "relaxation", "pleine conscience", "gestion du stress", "traumatisme"],
        "éducation": ["apprentissage", "enseignement", "pédagogie", "université", "formation", "compétence", "savoir", "connaissance", "méthode", "stratégie", "évaluation", "motivation", "curiosité", "créativité", "projet", "collaboration", "inclusion", "diversité"],
    }
    for theme, keywords in themes.items():
        if any(word in text.lower() for word in keywords):
            return theme
    return "inconnu"


In [6]:
# Créer la fonction à appliquer aux chunks de texte dans la colonne "type"
def detect_type(text):
    if "par exemple" in text.lower() or "exemple" in text.lower():
        return "exemple"
    elif "tu peux" in text.lower() or "il est conseillé" in text.lower():
        return "conseil"
    elif "c’est quoi" in text.lower() or "définition" in text.lower():
        return "définition"
    elif "comment" in text.lower() or "manière" in text.lower():
        return "méthode"
    elif "pourquoi" in text.lower() or "raison" in text.lower():
        return "explication"
    elif "qui" in text.lower() or "personne" in text.lower():
        return "personne"
    elif "quand" in text.lower() or "moment" in text.lower():
        return "temps"
    elif "où" in text.lower() or "lieu" in text.lower():
        return "lieu"
    elif "à toi" in text.lower() or "fais" in text.lower():
        return "action"
    elif "ressenti" in text.lower() or "émotion" in text.lower():
        return "émotion"
    else:
        return "inconnu"


In [7]:
# Créer la fonction à appliquer aux chunks de texte dans la colonne "niveau"

def estimate_level(text):
    word_count = len(text.split())
    if word_count < 80:
        return "facile"
    elif word_count < 150:
        return "intermédiaire"
    else:
        return "complexe"


In [8]:
# Appliquer les fonctions aux chunks de texte

df["theme"] = df["text"].apply(detect_theme)
df["type"] = df["text"].apply(detect_type)
df["niveau"] = df["text"].apply(estimate_level)

In [9]:
# Vérifier les résultats
df[["theme", "type", "niveau"]].head(10)

,theme,type,niveau
0,relations sociales,inconnu,facile
1,autisme,inconnu,facile
2,relations sociales,définition,complexe
3,émotions,méthode,complexe
4,relations sociales,action,complexe
5,émotions,personne,complexe
6,émotions,méthode,complexe
7,émotions,personne,complexe
8,émotions,méthode,complexe
9,inconnu,inconnu,facile


In [10]:
df.head()

,source,page,chunk_id,text,length,theme,niveau,type
0,100 idees pour enseigner les ha - Mehdi Liratni,2,100 idees pour enseigner les ha - Mehdi Liratn...,Dr Mehdi Liratni Préface du Pr René Pry 100 ID...,18,relations sociales,facile,inconnu
1,100 idees pour enseigner les ha - Mehdi Liratni,3,100 idees pour enseigner les ha - Mehdi Liratn...,La version numérique de cet ouvrage a été réal...,30,autisme,facile,inconnu
2,100 idees pour enseigner les ha - Mehdi Liratni,4,100 idees pour enseigner les ha - Mehdi Liratn...,SOMMAIRE Préface Remerciements INTRODUCTION HA...,171,relations sociales,complexe,définition
3,100 idees pour enseigner les ha - Mehdi Liratni,5,100 idees pour enseigner les ha - Mehdi Liratn...,CHAPITRE II COMPÉTENCES SOCLES DU COMPORTEMENT...,212,émotions,complexe,méthode
4,100 idees pour enseigner les ha - Mehdi Liratni,6,100 idees pour enseigner les ha - Mehdi Liratn...,Idée 26 Les 4 règles et objectifs importants d...,236,relations sociales,complexe,action


In [12]:
# Sauvegarder le DataFrame enrichi avec les métadonnées
df.to_csv("../data/ready_for_embedding/chunks_with_metadata.csv", index=False)
df = pd.read_csv("../data/ready_for_embedding/chunks_with_metadata.csv")
df.head()

,source,page,chunk_id,text,length,theme,niveau,type
0,100 idees pour enseigner les ha - Mehdi Liratni,2,100 idees pour enseigner les ha - Mehdi Liratn...,Dr Mehdi Liratni Préface du Pr René Pry 100 ID...,18,relations sociales,facile,inconnu
1,100 idees pour enseigner les ha - Mehdi Liratni,3,100 idees pour enseigner les ha - Mehdi Liratn...,La version numérique de cet ouvrage a été réal...,30,autisme,facile,inconnu
2,100 idees pour enseigner les ha - Mehdi Liratni,4,100 idees pour enseigner les ha - Mehdi Liratn...,SOMMAIRE Préface Remerciements INTRODUCTION HA...,171,relations sociales,complexe,définition
3,100 idees pour enseigner les ha - Mehdi Liratni,5,100 idees pour enseigner les ha - Mehdi Liratn...,CHAPITRE II COMPÉTENCES SOCLES DU COMPORTEMENT...,212,émotions,complexe,méthode
4,100 idees pour enseigner les ha - Mehdi Liratni,6,100 idees pour enseigner les ha - Mehdi Liratn...,Idée 26 Les 4 règles et objectifs importants d...,236,relations sociales,complexe,action
